In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1' - had to be updated to 3.3.2 to work.
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [78.0 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,586 kB]
Get:14 

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
homes_df.show()


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [4]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('home')



In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
    SELECT YEAR(date) as year, ROUND(AVG(price), 2) as avg_price
    FROM home
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""").show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) as avg_price
    FROM home
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""").show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) as avg_price
    FROM home
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""").show()




+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

import time

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2) as avg_view
    FROM home
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.6486489772796631 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home")


In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time1 = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2) as avg_view
    FROM home
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time1))


+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 1.6363303661346436 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
homes_df.write.partitionBy("date_built").parquet("path/to/save/home_sales.parquet")


In [13]:
# 11. Read the parquet formatted data.
homes_df1 = spark.read.parquet("path/to/save/home_sales.parquet")
homes_df1.head(5)

[Row(id='2ed8d509-7372-46d5-a9dd-9281a95467d4', date=datetime.datetime(2021, 8, 6, 0, 0), price=258710, bedrooms=3, bathrooms=3, sqft_living=1918, sqft_lot=9666, floors=1, waterfront=0, view=25, date_built=2015),
 Row(id='941bad30-eb49-4a78-b83a-87abb87a62db', date=datetime.datetime(2020, 5, 9, 0, 0), price=229896, bedrooms=3, bathrooms=3, sqft_living=2197, sqft_lot=8641, floors=1, waterfront=0, view=3, date_built=2015),
 Row(id='c797ca12-52cd-4b13-9338-183653619b11', date=datetime.datetime(2019, 6, 8, 0, 0), price=288650, bedrooms=2, bathrooms=3, sqft_living=2100, sqft_lot=10419, floors=2, waterfront=0, view=7, date_built=2015),
 Row(id='0cfe57f3-28c2-472c-9bc3-aaeac6807e62', date=datetime.datetime(2019, 10, 4, 0, 0), price=308313, bedrooms=3, bathrooms=3, sqft_living=1960, sqft_lot=9453, floors=2, waterfront=0, view=2, date_built=2015),
 Row(id='d715f295-2fbf-4e9a-a79b-b0437aed3777', date=datetime.datetime(2021, 5, 17, 0, 0), price=391574, bedrooms=3, bathrooms=2, sqft_living=1635, s

In [14]:
# 12. Create a temporary table for the parquet data.
homes_df.createOrReplaceTempView('home_parquet')


In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time2 = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2) as avg_view
    FROM home_parquet
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time2))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.2772212028503418 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home')


In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home')



False